Forested Protected Lands
===

This notebook overlays the DRWI protected lands and WeConservePA protected lands with the 2019 NLCD land use data using the ModelMyWatershed api. 

The goal of this notebook is to identify whether protected lands overlap with forested areas.

# Setup

## Imports

In [1]:
# Import packages
from pathlib import Path
import pandas as pd
import geopandas as gpd
import json
from modelmw_client import *

# Plotting packages
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from pyproj import CRS
from shapely.validation import make_valid
import colorcet as cc
from colorcet.plotting import swatch, swatches, sine_combs
import contextily as ctx

## File paths

In [2]:
# Find your current working directory, which should be folder for this notebook.
Path.cwd()

WindowsPath('C:/Users/clulay/OneDrive - LimnoTech/Documents/GitHub/pollution-assessment/stage2/Protected_Lands')

In [3]:
# Set your project directory to your local folder for your clone of this repository
project_path = Path.cwd().parent.parent
project_path

WindowsPath('C:/Users/clulay/OneDrive - LimnoTech/Documents/GitHub/pollution-assessment')

In [4]:
# Path to WeConservePA protected lands shapefile
wcpa_path = project_path / Path('stage2/Protected_Lands/WeConserve_pl_20220810.shp')
wcpa_path.exists()

True

## Private file paths
- Download from https://drive.google.com/drive/folders/1HKlnA3lC2xvKBQ1bg1J08gsEB3N9rp2V
- Save in `stage2/private` folder

In [5]:
# Path to FieldDoc protected lands export file
fielddoc_path = project_path / Path('stage2/private/protection_bmps_from_FieldDoc.parquet')
fielddoc_path.exists()

True

# Load data

## Protected Lands from WeConservePA

In [6]:
# Import weconservePA protected lands
wcpa_gdf = gpd.read_file(wcpa_path)

In [7]:
# Add data source tag
wcpa_gdf['data_source'] = 'weconservePA_2022'

## Protected Lands from FieldDoc

In [8]:
# Import FieldDoc protected lands 
fielddoc_gdf = gpd.read_parquet(fielddoc_path)

# Add a column to fielddoc_gdf with data source
fielddoc_gdf['data_source'] = 'fielddoc_2022-08-11'

## DRWI Geographies

In [9]:
# Import cluster and focus area geometries
cluster_gdf = gpd.read_parquet(project_path / Path('stage1/data/cluster_df.parquet'))  
focusarea_gdf = gpd.read_parquet(project_path / Path('stage1/data/fa_phase2_df.parquet'))

focusarea_gdf.cluster = focusarea_gdf.cluster.replace('Kirkwood Cohansey Aquifer', 'Kirkwood - Cohansey Aquifer') # update name for consistency with other files 
focusarea_gdf.set_index('name', inplace=True)

# Convert from geometries to geojson

In [21]:
items = [1,2,3,4]
sub_gdf = wcpa_gdf[wcpa_gdf['OBJECTID'].isin(items)]
wcpa_gdf.to_file(project_path / Path('stage2/Protected_Lands/wcpa.geojson'), driver="GeoJSON")
wcpa_json = json.load(open(project_path / Path('stage2/Protected_Lands/wcpa.geojson')))

C:\Users\clulay\.conda\envs\WikiSRAT\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [11]:
fielddoc_gdf.to_file(project_path / Path('stage2/private/fielddoc.geojson'), driver="GeoJSON")
fd_json = json.load(open(project_path / Path('stage2/private/fielddoc.geojson')))

C:\Users\clulay\.conda\envs\WikiSRAT\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


# MMW

In [12]:
staging_api_key = '82fb0268c9af8aeaac22c01910d344c3389586a2'

In [13]:
# Create an API user
mmw_run = ModelMyWatershedAPI(api_key=staging_api_key, use_staging=True)

In [52]:
# Plot weconservePA and fielddoc protected lands 
plt.rcParams["figure.figsize"] = (7,10)
wcpa_gdf = wcpa_gdf[wcpa_gdf['sitename']=='Estate of George E. Scheller']

In [55]:
def get_perc_forest_natural(geojson):
    perc_forest_dict = {'sitename': [],
                    'practice_type': [],
                    'mmw_area_m2': [],
                    'perc_forest': [],
                    'perc_natural': []}

    forest_lu = [41, 42, 43]

    # Include forest, shrub/scrub, grassland/herbaceous, and wetlands as "natural"
    natural_lu = [41, 42, 43, 52, 71, 90, 95]
    
    for item in geojson['features']:
        name = item['properties']['sitename']
        if (name != 'Arthur Weeks' and name != 'Caruso, Angelo J.' and name != 'Estate of George E. Scheller' 
            and name != 'Farm and Ranch Lands Protection Program (FRPP), Burlington, New Jersey'):
            print(name)
            practice_type = item['properties']['RECLASS2']

            item_lu_df = mmw_run.run_batch_analysis(
                [item['geometry']],
                mmw_run.land_endpoint.format("2019_2019"))

            total_area_m2 = item_lu_df['area'].sum()

            forest_df = item_lu_df[item_lu_df['nlcd'].isin(forest_lu)]
            forest_m2 = forest_df['area'].sum()

            natural_df = item_lu_df[item_lu_df['nlcd'].isin(natural_lu)]
            natural_m2 = natural_df['area'].sum()

            perc_forest = forest_m2/total_area_m2 * 100
            perc_natural = natural_m2/total_area_m2 * 100

            perc_forest_dict['sitename'].append(name)
            perc_forest_dict['practice_type'].append(practice_type)
            perc_forest_dict['mmw_area_m2'].append(total_area_m2)
            perc_forest_dict['perc_forest'].append(perc_forest)
            perc_forest_dict['perc_natural'].append(perc_natural)

    perc_forest_df = pd.DataFrame().from_dict(perc_forest_dict)
    return(perc_forest_df)

In [ ]:
get_perc_forest_natural(wcpa_json)

A & A Likanchuk Enterprises
A Massey And Mary G Moor Expansion Of The Mrs Grace Shorts Caulk District
A. Robert & Elizabeth Masten Forestland Preservation Area
A. Robert Masten District
Aaronson, Janet
Ackerman and Ackerman (former Elmer Trust)
Ackley, David B. & Nancy J. (Lot 4.03)
Adams Farm
Adamucci (1)
Addis, David & Priscilla
Agricultural Conservation Easement Program - Agricultural Land Easements (ACEP-ALE), Hunterdon, New Jersey
Agricultural Conservation Easement Program - Agricultural Land Easements (Acep-Ale), Kent, Delaware
Agricultural Conservation Easement Program - Agricultural Land Easements (Acep-Ale), New Castle, Delaware
Agricultural Conservation Easement Program - Agricultural Land Easements (ACEP-ALE), Salem, New Jersey
Agricultural Conservation Easement Program - Agricultural Land Easements (ACEP-ALE), Salem, New Jersey
Agricultural Conservation Easement Program - Agricultural Land Easements (Acep-Ale), Sussex, Delaware
Agricultural Easement
AJR
Alan & Joan Apgar
Al

In [20]:
wcpa_lu

,sitename,practice_type,mmw_area_m2,perc_forest,perc_natural
0,A & A Likanchuk Enterprises,Agricultural Easement,188438.510607,29.047619,29.047619
1,A Massey And Mary G Moor Expansion Of The Mrs ...,Agricultural Easement,200108.761614,0.000000,8.520179
2,A. Robert & Elizabeth Masten Forestland Preser...,Agricultural Easement,62817.072814,11.428571,97.142857
3,A. Robert Masten District,Agricultural Easement,680216.036057,5.013193,42.875989


In [ ]:
fig, ax = plt.subplots()
wcpa_gdf.geometry.boundary.plot(color=None,edgecolor='k',ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots()
fielddoc_gdf.geometry.boundary.plot(color=None,edgecolor='k',ax=ax)
plt.show()

In [ ]:
for item in wcpa_json['features']:
    print(item['geometry'])

In [ ]:
wcpa_run_dict = {}

#fielddoc_run_dict[test['practice_name']] = {}
for item in wcpa_json['features']:
    

    print(f'Starting...')
    # Set up request
    DATA = {'area_of_interest': item['geometry']}
    HEADERS = {'Authorization': f'Token {MMW_API_TOKEN}'}
    res_base = requests.post(f'{MMW_API_URL}/analyze/land/2019/',data=DATA, headers=HEADERS)

    print('  waiting for id ... ')
    time.sleep(10)
    print(res_base.json())
    #PREPARE_JOB_UUID = res_base.json()['job_uuid']
    #res_base = requests.get(f'{MMW_API_URL}/jobs/{PREPARE_JOB_UUID}/', headers=HEADERS)
    #time.sleep(10)
    
    print('  sending run...')
    